In [ ]:
package com.rc.tomato;

import org.opencv.core.Core;
import org.opencv.core.Mat;

public class Main {

    static {
        System.loadLibrary(Core.NATIVE_LIBRARY_NAME);
    }

    public static void main(String[] args) throws Exception {

        CameraCapture camera = new CameraCapture();
        TomatoDetector detector = new TomatoDetector();
        MotorController motor = new MotorController();

        System.out.println("Sistem Mobil RC Deteksi Kematangan Tomat Aktif");

        while (true) {
            Mat frame = camera.getFrame();
            if (frame.empty()) continue;

            String ripeness = detector.detectRipeness(frame);

            System.out.println("Hasil Deteksi: " + ripeness);

            if (ripeness.equals("MATANG (MERAH)")) {
                motor.stop();
            } else {
                motor.forward();
            }

            Thread.sleep(300);
        }
    }
}



package com.rc.tomato;

import org.opencv.core.Mat;
import org.opencv.videoio.VideoCapture;

public class CameraCapture {

    private VideoCapture camera;

    public CameraCapture() {
        camera = new VideoCapture(0);
    }

    public Mat getFrame() {
        Mat frame = new Mat();
        camera.read(frame);
        return frame;
    }
}

package com.rc.tomato;

import org.opencv.core.Mat;
import org.opencv.videoio.VideoCapture;

public class CameraCapture {

    private VideoCapture camera;

    public CameraCapture() {
        camera = new VideoCapture(0);
    }

    public Mat getFrame() {
        Mat frame = new Mat();
        camera.read(frame);
        return frame;
    }
}


package com.rc.tomato;

import org.opencv.core.*;
import org.opencv.imgproc.Imgproc;

public class TomatoDetector {

    public String detectRipeness(Mat frame) {

        Mat hsv = new Mat();
        Imgproc.cvtColor(frame, hsv, Imgproc.COLOR_BGR2HSV);

        double red = countColor(hsv, new Scalar(0,120,70), new Scalar(10,255,255));
        double green = countColor(hsv, new Scalar(35,50,50), new Scalar(85,255,255));

        String result = RipenessClassifier.classify(red, green);

        DetectionRepository.save(result, red, green);

        return result;
    }

    private double countColor(Mat hsv, Scalar lower, Scalar upper) {
        Mat mask = new Mat();
        Core.inRange(hsv, lower, upper, mask);
        return Core.countNonZero(mask);
    }
}



package com.rc.tomato;

public class RipenessClassifier {

    public static String classify(double red, double green) {

        if (red > green * 1.5) {
            return "MATANG (MERAH)";
        } else if (green > red * 1.5) {
            return "BELUM MATANG (HIJAU)";
        } else {
            return "SETENGAH MATANG";
        }
    }
}


package com.rc.tomato;

import com.pi4j.io.gpio.*;

public class MotorController {

    private final GpioController gpio = GpioFactory.getInstance();

    private final GpioPinDigitalOutput IN1 =
            gpio.provisionDigitalOutputPin(RaspiPin.GPIO_00, PinState.LOW);
    private final GpioPinDigitalOutput IN2 =
            gpio.provisionDigitalOutputPin(RaspiPin.GPIO_01, PinState.LOW);
    private final GpioPinDigitalOutput IN3 =
            gpio.provisionDigitalOutputPin(RaspiPin.GPIO_02, PinState.LOW);
    private final GpioPinDigitalOutput IN4 =
            gpio.provisionDigitalOutputPin(RaspiPin.GPIO_03, PinState.LOW);

    public void forward() {
        IN1.high(); IN2.low();
        IN3.high(); IN4.low();
    }

    public void stop() {
        IN1.low(); IN2.low();
        IN3.low(); IN4.low();
    }
}

